# Machine Learning Engineer Nanodegree
## Supervised Learning
## Project: Building a Student Intervention System

Welcome to the second project of the Machine Learning Engineer Nanodegree! In this notebook, some template code has already been provided for you, and it will be your job to implement the additional functionality necessary to successfully complete this project. Sections that begin with **'Implementation'** in the header indicate that the following block of code will require additional functionality which you must provide. Instructions will be provided for each section and the specifics of the implementation are marked in the code block with a `'TODO'` statement. Please be sure to read the instructions carefully!

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question X'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.  

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

### Question 1 - Classification vs. Regression
*Your goal for this project is to identify students who might need early intervention before they fail to graduate. Which type of supervised learning problem is this, classification or regression? Why?*

**Answer: This is a classification problem since the labels are categories or classes. Regression predicts quantitative values rather than labels. **

## Exploring the Data
Run the code cell below to load necessary Python libraries and load the student data. Note that the last column from this dataset, `'passed'`, will be our target label (whether the student graduated or didn't graduate). All other columns are features about each student.

In [2]:
# Import libraries
import numpy as np
import pandas as pd
from time import time
from sklearn.metrics import f1_score

# Read student data
student_data = pd.read_csv("student-data.csv")
print "Student data read successfully!"

Student data read successfully!


### Implementation: Data Exploration
Let's begin by investigating the dataset to determine how many students we have information on, and learn about the graduation rate among these students. In the code cell below, you will need to compute the following:
- The total number of students, `n_students`.
- The total number of features for each student, `n_features`.
- The number of those students who passed, `n_passed`.
- The number of those students who failed, `n_failed`.
- The graduation rate of the class, `grad_rate`, in percent (%).


In [3]:
# TODO: Calculate number of students
n_students=len(student_data.index)

# TODO: Calculate number of features
n_features = len(student_data.columns[:-1])

# TODO: Calculate passing students
n_passed = len(student_data[student_data.passed=="yes"])

# TODO: Calculate failing students
n_failed = len(student_data[student_data.passed=="no"])

# TODO: Calculate graduation rate
grad_rate = 100*float(n_passed)/float(n_students)

# Print the results
print "Total number of students: {}".format(n_students)
print "Number of features: {}".format(n_features)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Graduation rate of the class: {:.2f}%".format(grad_rate)

Total number of students: 395
Number of features: 30
Number of students who passed: 265
Number of students who failed: 130
Graduation rate of the class: 67.09%


## Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Run the code cell below to separate the student data into feature and target columns to see if any features are non-numeric.

In [4]:
# Extract feature columns
feature_cols = list(student_data.columns[:-1])

# Extract target column 'passed'
target_col = student_data.columns[-1] 

# Show the list of columns
print "Feature columns:\n{}".format(feature_cols)
print "\nTarget column: {}".format(target_col)

# Separate the data into feature data and target data (X_all and y_all, respectively)
X_all = student_data[feature_cols]
y_all = student_data[target_col]

# Show the feature information by printing the first five rows
print "\nFeature values:"
print X_all.head()

Feature columns:
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']

Target column: passed

Feature values:
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...       

### Preprocess Feature Columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation. Run the code cell below to perform the preprocessing routine discussed in this section.

In [5]:
def preprocess_features(X):
    ''' Preprocesses the student data and converts non-numeric binary variables into
        binary (0/1) variables. Converts categorical variables into dummy variables. '''
    
    # Initialize new output DataFrame
    output = pd.DataFrame(index = X.index)

    # Investigate each feature column for the data
    for col, col_data in X.iteritems():
        
        # If data type is non-numeric, replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])

        # If data type is categorical, convert to dummy variables
        if col_data.dtype == object:
            # Example: 'school' => 'school_GP' and 'school_MS'
            col_data = pd.get_dummies(col_data, prefix = col)  
        
        # Collect the revised columns
        output = output.join(col_data)
    
    return output

X_all = preprocess_features(X_all)
print "Processed feature columns ({} total features):\n{}".format(len(X_all.columns), list(X_all.columns))

Processed feature columns (48 total features):
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Implementation: Training and Testing Data Split
So far, we have converted all _categorical_ features into numeric values. For the next step, we split the data (both features and corresponding labels) into training and test sets. In the following code cell below, you will need to implement the following:
- Randomly shuffle and split the data (`X_all`, `y_all`) into training and testing subsets.
  - Use 300 training points (approximately 75%) and 95 testing points (approximately 25%).
  - Set a `random_state` for the function(s) you use, if provided.
  - Store the results in `X_train`, `X_test`, `y_train`, and `y_test`.

In [6]:
# TODO: Import any additional functionality you may need here
from sklearn.model_selection import train_test_split
# TODO: Set the number of training points
num_train = 300

# Set the number of testing points
num_test = X_all.shape[0] - num_train

# TODO: Shuffle and split the dataset into the number of training and testing points above
features_train, features_test, labels_train, labels_test = train_test_split(X_all, y_all, test_size=num_test, random_state=0)
X_train = features_train
X_test = features_test
y_train = labels_train
y_test = labels_test

# Show the results of the split
print "Training set has {} samples.".format(X_train.shape[0])
print "Testing set has {} samples.".format(X_test.shape[0])
X_train.head()

Training set has 300 samples.
Testing set has 95 samples.


,school_GP,school_MS,sex_F,sex_M,age,address_R,address_U,famsize_GT3,famsize_LE3,Pstatus_A,...,higher,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences
63,1.0,0.0,1.0,0.0,16,0.0,1.0,1.0,0.0,0.0,...,1,1,0,3,4,4,2,4,4,2
245,1.0,0.0,0.0,1.0,16,0.0,1.0,1.0,0.0,0.0,...,1,1,0,4,3,3,1,1,4,6
154,1.0,0.0,1.0,0.0,17,0.0,1.0,1.0,0.0,0.0,...,1,0,1,4,2,1,1,1,4,0
311,1.0,0.0,1.0,0.0,19,0.0,1.0,1.0,0.0,0.0,...,0,1,1,3,4,1,1,1,2,20
81,1.0,0.0,0.0,1.0,15,0.0,1.0,1.0,0.0,0.0,...,1,1,0,5,3,2,1,2,5,4


## Training and Evaluating Models
In this section, you will choose 3 supervised learning models that are appropriate for this problem and available in `scikit-learn`. You will first discuss the reasoning behind choosing these three models by considering what you know about the data and each model's strengths and weaknesses. You will then fit the model to varying sizes of training data (100 data points, 200 data points, and 300 data points) and measure the F<sub>1</sub> score. You will need to produce three tables (one for each model) that shows the training set size, training time, prediction time, F<sub>1</sub> score on the training set, and F<sub>1</sub> score on the testing set.

**The following supervised learning models are currently available in** [`scikit-learn`](http://scikit-learn.org/stable/supervised_learning.html) **that you may choose from:**
- Gaussian Naive Bayes (GaussianNB)
- Decision Trees
- Ensemble Methods (Bagging, AdaBoost, Random Forest, Gradient Boosting)
- K-Nearest Neighbors (KNeighbors)
- Stochastic Gradient Descent (SGDC)
- Support Vector Machines (SVM)
- Logistic Regression

### Question 2 - Model Application
*List three supervised learning models that are appropriate for this problem. For each model chosen*
- Describe one real-world application in industry where the model can be applied. *(You may need to do a small bit of research for this — give references!)* 
- What are the strengths of the model; when does it perform well? 
- What are the weaknesses of the model; when does it perform poorly?
- What makes this model a good candidate for the problem, given what you know about the data?

**Answer: **
Since this is a classification problem, I chose three classification methods that I want to understand better and that I think will perform well for the purposes of student intervention. I wanted to try to different types of classifications methods to start. 

<ul style="list-style-type:circle">

  <li>**K-Nearest Neighbors (KNeighbors)**</li>
  <p> KNN can be used for digit recognition. In the example in the link below, handwritten digits are normalized and then the pixels are simplified. The original bitmaps are 32X32 but simplified to an input matrix of 8x8 which reduces the dimensionality to 64 instead of 1024. The classifier is tuned using 10-fold cross validation. The selected kNN classifier, with k = 4 closest data points and d = 36 derived features,  obtains an error rate of 1.89% on the testing data set. Teh classifier was also With the testing data set, resulting in an error rate of 1.4%.   </p>
  <br>
   <a href="http://analytics.ncsu.edu/sesug/2012/SD-09.pdf">  KNN CLASSIFICATION IN OPTICAL DIGITS </a>
  <br>
      <p>K-Nearest Neighbors is considered a baseline algorithm, meaning often times we use KNN to compare other algorithms to. It is a simple algorithm that is surprisingly accurate and hard to beat. Since KNN uses a distance metric, it is less affective in cases where density is more important than distance. </p>
  <p> The pros of K-Nearest Neighbors are that it usually has high accuracy, is insensitive to outliers, and can be used with no assumptions about data. Those are huge! However, the cons are that it is computationally expensive and requires a lot of memory. But, it's not so bad if your data set isn't huge and you have the computer power to handle it.  </p> 
  <p> Thus, KNN has some great pros and may or may not have serious cons. This is why it is such a hard to beat algorithm. In many cases, the results are easily interpretable.  For these reasons, I wanted to include KNN in my study, to use as a baseline and to have that baseline be easy to interpret. However, since many of the features are turned into binary data, their weight (due to scaling) is not as high as other features that are larger such as age. Still, KNN is a common baseline. </p> 
  <br>
  <br>
  <li>**Support Vector Machines (SVM)**</li>
  <p>Support Vector Machines can be used to prevent and detect fraudulent credit card activity. Supervised or unsupervised methods can be used, and the types of data includes typically spending, geographical location, or Merchant Catergory Codes. SVM models built in this study use four different kernel functions: polynomial, sigmoid, radial basis and linear kernel functions. However, in the study below, the Decision Tree Methods outperformed the SVMs, still, SVMs did approach the accuracy of the Decision Trees. </p>
  <br>
  <a href="http://www.iaeng.org/publication/IMECS2011/IMECS2011_pp442-447.pdf">Detecting Credit Card Fraud by Decision Trees and Support Vector Machines </a>
  
  
  <br>
  <p>Support Vector Machines (SVMs) are essentially the same as Logistic Regression with a linear kernel. Even though Logistic Regresesion can handle non-linear kernels, SVM is generally better for problems that are not linearly separable. SVMs handle high dimensional space very well, in particular in spaces where there are more dimensions than samples (but not too many more). However, they can be inefficient to train with "industry scale" datasets and only work when there is pretty clear margin of separation (not much overlap/noise). </p>
<p> I wanted to use SVMs because I think it is good to use methods with simple stories that can be understood in non-technical people can understand. Support Vector Machines are easy to understand with a simple example. I believe it is good to try SVMS in case the data has a clear margin of separation. </p>

<br>
<br>

    <li>** AdaBoost**</li>
    <p>A classic machine learning problem is the classification of email as spam or not spam. In the article below, they used AdaBoost, with decision stumps as its base learner to determine spam or not spam. Researchers in 1999 used C4.5 decision trees as weak learners and reported high accuracy but slow execution time and they ended up recommendeding SVM instead. In 2001, researchers used decision stumps instead; with a single node and found the algorithm ran faster. In this article, candidate stumps are simply the presence or absence of a word in an email, which does scale very well with many key words. This article suggests that the AdaBoost algorithm, using decision stumps, is inferior in terms of both accuracy and speed to the Naive Bayes algorithm for this problem. With f = 3 and T = 3 on the ``tiny” data set, AdaBoost obtained only 86.7 percent accuracy, compared
with 95.3 percent for Naive Bayes. The speed and performance could be improved by developing
better heuristics, in particular, how to choose the weak learners. </p>
<br>
    <a href="http://nlp.stanford.edu/courses/cs224n/2003/fp/tyronen/report.pdf">Adaboost article from Stanford</a>
<br>
<p> 
Another study showed that Adaboost was very effective in page ranking algorithm. The use an Adaboost Subtrance Classification Model which uses a tree-like structure to illustrate relationships between web content. The paper demostrates several boosting algorithms that outperform the commonly used data mining methods. 
</p> 

<br>
    <a href="http://idosi.org/wasj/wasj34(1)16/18.pdf">Adaboost article from World Applied Sciences Journal</a>
<br>
    <p>An AdaBoost classifier is a linear combination of weak learners that begins by fitting a classifier on the original dataset and then fits additional copies of the classifier on the same dataset but where the weights of incorrectly classified instances are adjusted such that subsequent classifiers focus more on difficult cases.  If the model fails at some point, you can determine at which weak learner it failed. This means you can manually tune the models to improve performance. Since with Machine Learning, overfitting is a common problem, Adaboost allows you to fine tune in addition to the "boosting" to avoid overfitting. However, Adaboost is fairly resistant to overfitting anyway. The default classifier, which will be used to start, is the Decision Tree Classifier. Decision Trees are a great place to start, because they are another algorithm that is easy to understand, by the lay person, with simple examples. I wanted to give Adaboost a try because it is considered one of the best out of the box classifiers. </p>
    
<p> While the first article mentions that Naive Bayes performed better than Adaboost, I also tried Naive Bayes on the student intevervention data and its performance was inferior to the other two methods I tried initially. Thus, I choose Adaboost in favor of Naive Bayes. </p>


</ul>




### Setup
Run the code cell below to initialize three helper functions which you can use for training and testing the three supervised learning models you've chosen above. The functions are as follows:
- `train_classifier` - takes as input a classifier and training data and fits the classifier to the data.
- `predict_labels` - takes as input a fit classifier, features, and a target labeling and makes predictions using the F<sub>1</sub> score.
- `train_predict` - takes as input a classifier, and the training and testing data, and performs `train_clasifier` and `predict_labels`.
 - This function will report the F<sub>1</sub> score for both the training and testing data separately.

In [8]:
def train_classifier(clf, X_train, y_train):
    ''' Fits a classifier to the training data. '''
    
    # Start the clock, train the classifier, then stop the clock
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    
    # Print the results
    print "Trained model in {:.4f} seconds".format(end - start)

    
def predict_labels(clf, features, target):
    ''' Makes predictions using a fit classifier based on F1 score. '''
    
    # Start the clock, make predictions, then stop the clock
    start = time()
    y_pred = clf.predict(features)
    end = time()
    
    # Print and return results
    print "Made predictions in {:.4f} seconds.".format(end - start)
    return f1_score(target.values, y_pred, pos_label='yes')


def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' Train and predict using a classifer based on F1 score. '''
    
    # Indicate the classifier and the training set size
    print "Training a {} using a training set size of {}. . .".format(clf.__class__.__name__, len(X_train))
    
    # Train the classifier
    train_classifier(clf, X_train, y_train)
    
    # Print the results of prediction for both training and testing
    print "F1 score for training set: {:.4f}.".format(predict_labels(clf, X_train, y_train))
    print "F1 score for test set: {:.4f}.".format(predict_labels(clf, X_test, y_test))

### Implementation: Model Performance Metrics
With the predefined functions above, you will now import the three supervised learning models of your choice and run the `train_predict` function for each one. Remember that you will need to train and predict on each classifier for three different training set sizes: 100, 200, and 300. Hence, you should expect to have 9 different outputs below — 3 for each model using the varying training set sizes. In the following code cell, you will need to implement the following:
- Import the three supervised learning models you've discussed in the previous section.
- Initialize the three models and store them in `clf_A`, `clf_B`, and `clf_C`.
 - Use a `random_state` for each model you use, if provided.
 - **Note:** Use the default settings for each model — you will tune one specific model in a later section.
- Create the different training set sizes to be used to train each model.
 - *Do not reshuffle and resplit the data! The new training points should be drawn from `X_train` and `y_train`.*
- Fit each model with each training set size and make predictions on the test set (9 in total).  
**Note:** Three tables are provided after the following code cell which can be used to store your results.

In [17]:
# TODO: Import the three supervised learning models from sklearn
# from sklearn import model_A
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
# from sklearn import model_B
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
# from skearln import model_C
from sklearn.svm import SVC
from sklearn.utils import resample

# TODO: Initialize the three models
clf_A = KNeighborsClassifier()
#clf_A = GaussianNB()
clf_B = AdaBoostClassifier(random_state=0)
#clf_B = DecisionTreeClassifier(random_state=0)
clf_C = SVC(random_state=0)

clustering_algorithms=[clf_A, clf_B, clf_C]

# TODO: Set up the training set sizes

num_not_train=[200,100,0]


# TODO: Execute the 'train_predict' function for each classifier and each training set size

for clf_name in clustering_algorithms:
    for num in num_not_train:
        num_test = X_train.shape[0] - num
        X_train_subset=resample(X_train, n_samples=num_test,random_state=0)
        y_train_subset=resample(y_train, n_samples=num_test, random_state=0)
        train_predict(clf_name, X_train_subset, y_train_subset, X_test, y_test)
        
print "F1 score for predicting all \"yes\" on test set: {:.4f}".format(
    f1_score(y_test, ['yes']*len(y_test), pos_label='yes', average='binary'))


Training a KNeighborsClassifier using a training set size of 100. . .
Trained model in 0.0007 seconds
Made predictions in 0.0020 seconds.
F1 score for training set: 0.8662.
Made predictions in 0.0020 seconds.
F1 score for test set: 0.7586.
Training a KNeighborsClassifier using a training set size of 200. . .
Trained model in 0.0006 seconds
Made predictions in 0.0030 seconds.
F1 score for training set: 0.8987.
Made predictions in 0.0020 seconds.
F1 score for test set: 0.7500.
Training a KNeighborsClassifier using a training set size of 300. . .
Trained model in 0.0007 seconds
Made predictions in 0.0054 seconds.
F1 score for training set: 0.9052.
Made predictions in 0.0025 seconds.
F1 score for test set: 0.7313.
Training a AdaBoostClassifier using a training set size of 100. . .
Trained model in 0.1338 seconds
Made predictions in 0.0042 seconds.
F1 score for training set: 1.0000.
Made predictions in 0.0040 seconds.
F1 score for test set: 0.6970.
Training a AdaBoostClassifier using a trai

### Tabular Results
Edit the cell below to see how a table can be designed in [Markdown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet#tables). You can record your results from above in the tables provided.

** Classifer 1 - KNeighborsClassifier**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |        0.0006           |         0.0019         |      0.8662      |     0.7586      |
| 200               |        0.0006           |         0.0030         |      0.8987      |     0.7500      |
| 300               |        0.0007           |         0.0053         |      0.9052      |     0.7313      |

** Classifer 2 - AdaBoostClassifier**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |      0.1245             |        0.0041          |      1.0000      |     0.6970      |
| 200               |      0.1272             |        0.0048          |      0.9800      |     0.7500      |
| 300               |      0.1266             |        0.0072          |      0.9217      |     0.7770      |

** Classifer 3 - Support Vector Classifier**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |         0.0010          |       0.0006           |      0.9012      |      0.7733     |
| 200               |         0.0022          |       0.0016           |      0.9193      |      0.7891     |
| 300               |         0.0043          |       0.0034           |      0.9333      |      0.7832     |



## Choosing the Best Model
In this final section, you will choose from the three supervised learning models the *best* model to use on the student data. You will then perform a grid search optimization for the model over the entire training set (`X_train` and `y_train`) by tuning at least one parameter to improve upon the untuned model's F<sub>1</sub> score. 

### Question 3 - Choosing the Best Model
*Based on the experiments you performed earlier, in one to two paragraphs, explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?*

**Answer: **  I recommend using the the Adaboost Classifier. While the Support Vector Classifier has slightly higher F1 scores for the testing data, Support Vector algorithms are considered not that practical in the real world unless the data is particularly non-overlapping. In other words, we would have to have very distinct groups in order for Support Vector algorithms to be reliable. Adaboost is a useful algorithm since it can easily be tuned in many ways to avoid common problems such as overfitting and to improve accuracy.  

Another issue is that the Adaboost training time is about 100 times longer. While the F1 scores are so close, you could use this training time to break the tie. However, I believe that Adaboost has much more potential to improve the F1 score with tuning. Additionally, the Adaboost's training time seems to be fairly consistent across different training set sizes, whereas the Support Vector Classifier's training times double with each additional 100 data points added. This trent could continue on larger, real datasets. 

While K-Nearest Neighbors is supposed to be hard to beat, both the Adaboost and SVM methods beat it here. 

### Question 4 - Model in Layman's Terms
*In one to two paragraphs, explain to the board of directors in layman's terms how the final model chosen is supposed to work. Be sure that you are describing the major qualities of the model, such as how the model is trained and how the model makes a prediction. Avoid using advanced mathematical or technical jargon, such as describing equations or discussing the algorithm implementation.*

**Answer: ** 
Since the goal is to identify students who might need early intervention before they fail to graduate, the Adaboost algorithm will use a collection of models to predict whether the students are in the "yes" or "no" category for neededing intervention. 

The method of classification called Adaboost can build a model by putting together several less effective models and averaging their results to make a better prediction. We wouldn't want to use one of the models by themselves because they do not perform well enough. But it has been shown that Adaboost can improve the performance by putting these "weak learners," as they are called, together into one averaged model. The model can be improved upon with a set of data reserved for "training" the model. The first version of the Adaboost model is run and any of the models within Adaboost that predict incorrectly are weighted lower and any models that predict correctly are weighted higher in the final average. 

You can also do a "tuning" phase which also helps you decide which models to use to put together.  I will compare the Decision Tree Classifier, which is usually used with Adaboost, as well  Support Vector Classifier since it did pretty well in the preliminary testing. Using one of these models at a time, they will be tested across different numbers of models. This will help me choose the best parameters for the collection of models to be used. 

### Implementation: Model Tuning
Fine tune the chosen model. Use grid search (`GridSearchCV`) with at least one important parameter tuned with at least 3 different values. You will need to use the entire training set for this. In the code cell below, you will need to implement the following:
- Import [`sklearn.grid_search.GridSearchCV`](http://scikit-learn.org/0.17/modules/generated/sklearn.grid_search.GridSearchCV.html) and [`sklearn.metrics.make_scorer`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html).
- Create a dictionary of parameters you wish to tune for the chosen model.
 - Example: `parameters = {'parameter' : [list of values]}`.
- Initialize the classifier you've chosen and store it in `clf`.
- Create the F<sub>1</sub> scoring function using `make_scorer` and store it in `f1_scorer`.
 - Set the `pos_label` parameter to the correct value!
- Perform grid search on the classifier `clf` using `f1_scorer` as the scoring method, and store it in `grid_obj`.
- Fit the grid search object to the training data (`X_train`, `y_train`), and store it in `grid_obj`.

In [19]:
# TODO: Import 'GridSearchCV' and 'make_scorer'
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer

# TODO: Create the parameters list you wish to tune
#parameters = {'kernel':('linear', 'poly', 'rbf'), 'C':[1, 10]}
parameters = {'base_estimator':(DecisionTreeClassifier(max_depth=2), SVC(kernel='linear')), 'n_estimators':[25, 50, 200]}

# TODO: Initialize the classifier
svc = AdaBoostClassifier(algorithm='SAMME', random_state=0)
#svc = SVC()

# TODO: Make an f1 scoring function using 'make_scorer' 
f1_scorer = make_scorer(f1_score, pos_label='yes')

# TODO: Perform grid search on the classifier using the f1_scorer as the scoring method
grid_obj = GridSearchCV(svc, parameters, scoring=f1_scorer)

# TODO: Fit the grid search object to the training data and find the optimal parameters
grid_obj = grid_obj.fit(X_train, y_train)

# Get the estimator
clf = grid_obj.best_estimator_

print clf 

# Report the final F1 score for training and testing after parameter tuning
print "Tuned model has a training F1 score of {:.4f}.".format(predict_labels(clf, X_train, y_train))
print "Tuned model has a testing F1 score of {:.4f}.".format(predict_labels(clf, X_test, y_test))

AdaBoostClassifier(algorithm='SAMME',
          base_estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
          learning_rate=1.0, n_estimators=25, random_state=0)
Made predictions in 0.0455 seconds.
Tuned model has a training F1 score of 0.8132.
Made predictions in 0.0152 seconds.
Tuned model has a testing F1 score of 0.8085.


### Question 5 - Final F<sub>1</sub> Score
*What is the final model's F<sub>1</sub> score for training and testing? How does that score compare to the untuned model?*

**Answer: **

The tuned Adaboost model used a base_estimator of SCV(kernel='linear') rather than the default DecisionTreeClassifier. The tuned model has a training F1 score of 0.8132 and a testing F1 score of 0.8085. This is notably better than random guessing, which had an F1 score of 

The untuned  AdaBoostClassifier using a training set has F1 scores ranging from 0.92-1.00 and testing scores ranging from  0.6970-0.7770. Thus, the testing score improved. 

Also, the training score seems to have converged with the testing score which is a sign that overfitting was occurring, but was removed. 




> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  
**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.